## Workflow 2, Module 2d

In [1]:
#To make nicer looking outputs
from IPython.core.display import display, HTML
import requests
import pandas as pd
import os
import sys

#Load some functions for parsing quick output
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
        sys.path.append(module_path)
from gg_functions import parse_answer, get_view_url, expand, quick

robokop='robokop.renci.org' 

In [2]:
def create_question(disease_id,prop):
    return {
    "machine_question": {
        "nodes": [
            {
                "id": "n0",
                "type": "chemical_substance",
                prop: True
            },
            {
                "id": "n1",
                "type": "gene"
            },
            {
                "id": "n2",
                "type": "disease",
                "curie": disease_id
            }
        ],
        "edges": [
            {
                "id": "e0",
                "source_id": "n1",
                "target_id": "n0",
                "type": ["increases_degradation_of","decreases_response_to"]
            }
        ]
    }
}


In [3]:
from scipy.stats import hypergeom
from collections import defaultdict
#Read in the list of known property counts
property_counts = pd.read_csv('../../examples/chemprops.txt','	')

def get_contributing_chemicals(disease_id):
    # Get the chemicals that contribute to the disease
    exp_chemicals = expand('disease',disease_id,'chemical_substance',predicate='contributes_to',direction='in')
    return [x for x in exp_chemicals['knowledge_graph']['nodes'] if x['type'] == 'chemical_substance']

def get_property_counts(bad_chems):
    prop_counts = defaultdict(int)
    for n in bad_chems:
        for p in n:
            prop_counts[p] += 1
            items = list(prop_counts.items())
            count_frame = pd.DataFrame.from_records(items,columns=['property','count_in_subset'])
    return count_frame

#in scipy:
#The hypergeometric distribution models drawing objects from a bin. 
#M is the total number of objects, 
#n is total number of Type I objects. 
#The random variate represents the number of Type I objects in N drawn without replacement from the total population.

total_chemical_count=355413 # M above

def calc_enrich_p(x,n,subset_count):
    return hypergeom.sf(x-1, total_chemical_count, n, subset_count)

def execute_example(disease_id,disease_name):
    display(HTML(f'<h3>{disease_name}</h3>'))
    bad_chems = get_contributing_chemicals(disease_id)
    if len(bad_chems) ==0:
        print('No contributing chemicals')
        return
    count_frame = get_property_counts(bad_chems)
    df = pd.merge(count_frame, property_counts,on='property',how='inner')
    for p in ['drug','pharmaceutical','application','pharmacological_role','biological_role','entity',
              'continuant','role','mass','monoisotopic_mass','charge','chemical_role','biochemical_role']:
        df = df[ df['property'] != p]
    df['enrichment_p'] = df.apply(lambda x: calc_enrich_p(x['count_in_subset'], x['count'],len(bad_chems)), axis=1)
    df.sort_values(by='enrichment_p',inplace=True)
    propname = df.iloc[0,0]
    print('Most enriched property:',propname)
    q = create_question(disease_id,propname)
    answers=quick(q)
    if len(answers['answers']) == 0:
        print("No answers found")
        return
    view_url = get_view_url(answers)
    display(HTML(f'<a href={view_url}>View Answer in ROBOKOP</a>'))
    answer_frame = parse_answer(answers,node_list=['n0','n1'])
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(answer_frame)


In [4]:
execute_example("MONDO:0019391","fanconi anemia")

Most enriched property: mutagen
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:21759,N-methyl-N'-nitro-N-nitrosoguanidine,HGNC:11998,TP53,0.450783
1,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.406247
2,CHEBI:4305,dacarbazine,HGNC:7059,MGMT,0.403342
3,CHEBI:21759,N-methyl-N'-nitro-N-nitrosoguanidine,HGNC:7059,MGMT,0.394689
4,CHEBI:4305,dacarbazine,HGNC:12680,VEGFA,0.394153
5,CHEBI:50102,N-methyl-N-nitrosourea,HGNC:7059,MGMT,0.386600
6,CHEBI:75270,nimustine,HGNC:7059,MGMT,0.356542
7,CHEBI:72564,temozolomide,HGNC:587,APEX1,0.355948
8,CHEBI:3423,carmustine,HGNC:587,APEX1,0.355858
9,CHEBI:28830,chlorambucil,HGNC:4632,GSTM1,0.355842


In [5]:
execute_example("MONDO:0019600","xeroderma pigmentosum")

Most enriched property: cross_linking_reagent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:18358,methoxsalen,HGNC:1046,BHLHE40,0.232464


In [6]:
execute_example("MONDO:0016006","cockayne syndrome")

No contributing chemicals


In [7]:
execute_example("MONDO:0015797","UV-sensitive syndrome")

No contributing chemicals


In [8]:
execute_example("MONDO:0011125","Triciothiodystrophy")

No contributing chemicals


In [9]:
execute_example("MONDO:0001076","glucose intolerance")

Most enriched property: central_nervous_system_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16236,ethanol,HGNC:404,ALDH2,0.316605
1,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.314928
2,CHEBI:27958,cocaine,HGNC:3796,FOS,0.312421
3,CHEBI:3766,clozapine,HGNC:2596,CYP1A2,0.312168
4,CHEBI:9753,troglitazone,HGNC:9237,PPARGC1A,0.312055
5,CHEBI:8107,phenytoin,HGNC:1516,CAT,0.311990
6,CHEBI:27958,cocaine,HGNC:1033,BDNF,0.311956
7,CHEBI:2679,amphetamine,HGNC:7955,NPY,0.311556
8,CHEBI:16236,ethanol,HGNC:13633,ADIPOQ,0.311485
9,CHEBI:16236,ethanol,HGNC:1516,CAT,0.310948


In [10]:
execute_example("MONDO:0010086","sudden infant death syndrome")

Most enriched property: antiviral_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:10110,zidovudine,HGNC:11180,SOD2,0.309734
1,CHEBI:10110,zidovudine,HGNC:1516,CAT,0.309718
2,CHEBI:10110,zidovudine,HGNC:11830,TK1,0.277381
3,CHEBI:175901,gemcitabine,HGNC:651,AREG,0.232179
4,CHEBI:10110,zidovudine,HGNC:12816,XPC,0.232151
5,CHEBI:175901,gemcitabine,HGNC:11283,SRC,0.215141
6,CHEBI:60761,floxuridine,HGNC:882,ATR,0.214941
7,CHEBI:60761,floxuridine,HGNC:12441,TYMS,0.214909
8,CHEBI:175901,gemcitabine,HGNC:10773,SFN,0.214901
9,CHEBI:60761,floxuridine,HGNC:12828,XRCC1,0.214879


In [11]:
execute_example("MONDO:0008767","neuronal ceroid lipofuscinosis 3")

No contributing chemicals


In [12]:
execute_example("MONDO:0009757","Niemann-Pick disease, type C1")

No contributing chemicals


In [13]:
execute_example("MONDO:0010940","inherited susceptibility to asthma")

No contributing chemicals


In [14]:
execute_example("MONDO:0007319","familial calcium pyrophosphate deposition")

No contributing chemicals


In [15]:
execute_example("MONDO:0020087","genetic lipodystrophy")

No contributing chemicals


In [16]:
execute_example("MONDO:0007455","diabetes mellitus, noninsulin-dependent")

No contributing chemicals


In [17]:
execute_example("MONDO:0007191","Behcet disease")

Most enriched property: immunomodulator
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:46345,5-fluorouracil,HGNC:3012,DPYD,0.628871
1,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.395461
2,CHEBI:46345,5-fluorouracil,HGNC:11998,TP53,0.371267
3,CHEBI:17823,calcitriol,HGNC:2602,CYP24A1,0.359965
4,CHEBI:28830,chlorambucil,HGNC:4632,GSTM1,0.358454
5,CHEBI:41879,dexamethasone,HGNC:990,BCL2,0.356246
6,CHEBI:175901,gemcitabine,HGNC:11283,SRC,0.355976
7,CHEBI:175901,gemcitabine,HGNC:10773,SFN,0.355743
8,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.329019
9,CHEBI:44185,methotrexate,HGNC:7436,MTHFR,0.314965


In [18]:
execute_example("MONDO:0005100","systemic sclerosis")

Most enriched property: antineoplastic_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:46345,5-fluorouracil,HGNC:3012,DPYD,0.628871
1,CHEBI:45863,paclitaxel,HGNC:11998,TP53,0.540813
2,CHEBI:15367,all-trans-retinoic acid,HGNC:2603,CYP26A1,0.452246
3,CHEBI:46345,5-fluorouracil,HGNC:11998,TP53,0.451103
4,CHEBI:45863,paclitaxel,HGNC:11364,STAT3,0.450864
5,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.450380
6,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.420899
7,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.405370
8,CHEBI:41774,tamoxifen,HGNC:3236,EGFR,0.396267
9,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.395480


In [19]:
execute_example("MONDO:0010602","hemophilia A")

Most enriched property: HIV_protease_inhibitor
Return Status: 200
No answers found


In [20]:
execute_example("MONDO:0006507","hereditary hemochromatosis")

Most enriched property: micronutrient
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:30052,copper(0),HGNC:870,ATP7B,0.365852
1,CHEBI:28694,copper atom,HGNC:870,ATP7B,0.365852
2,CHEBI:18248,iron atom,HGNC:4553,GPX1,0.340828
3,CHEBI:15422,ATP,HGNC:8537,P2RX7,0.321788
4,CHEBI:15379,dioxygen,HGNC:2596,CYP1A2,0.320006
5,CHEBI:30052,copper(0),HGNC:9449,PRNP,0.315387
6,CHEBI:28694,copper atom,HGNC:9449,PRNP,0.315386
7,CHEBI:26689,singlet dioxygen,HGNC:2596,CYP1A2,0.286143
8,CHEBI:27140,triplet dioxygen,HGNC:2596,CYP1A2,0.286143
9,CHEBI:17823,calcitriol,HGNC:2602,CYP24A1,0.281287


In [21]:
execute_example("MONDO:0019599","primary lipodystrophy")

No contributing chemicals


In [22]:
execute_example("MONDO:0015547","genetic dementia")

No contributing chemicals


In [23]:
execute_example("MONDO:0015978","functional neutrophil defect")

No contributing chemicals


In [24]:
execute_example("MONDO:0007186","gastroesophageal reflux disease")

Most enriched property: neurotransmitter_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16240,hydrogen peroxide,HGNC:1516,CAT,0.598193
1,CHEBI:16240,hydrogen peroxide,HGNC:11998,TP53,0.539605
2,CHEBI:16240,hydrogen peroxide,HGNC:4638,GSTP1,0.494931
3,CHEBI:16240,hydrogen peroxide,HGNC:4553,GPX1,0.404164
4,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.394794
5,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.359530
6,CHEBI:41879,dexamethasone,HGNC:990,BCL2,0.356016
7,CHEBI:18243,dopamine,HGNC:6834,MAOB,0.320748
8,CHEBI:16240,hydrogen peroxide,HGNC:16753,PRDX6,0.320390
9,CHEBI:16240,hydrogen peroxide,HGNC:4629,GSTA4,0.314808


In [25]:
execute_example("MONDO:0009061","cystic fibrosis")

Most enriched property: cardiovascular_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.359606
1,CHEBI:8093,phenylephrine,HGNC:7876,NOS3,0.358576
2,CHEBI:8093,phenylephrine,HGNC:7873,NOS2,0.357237
3,CHEBI:27958,cocaine,HGNC:983,BCHE,0.356913
4,CHEBI:18243,dopamine,HGNC:6834,MAOB,0.320748
5,CHEBI:16480,nitric oxide,HGNC:333,AGT,0.314953
6,CHEBI:34665,debrisoquin,HGNC:2625,CYP2D6,0.313600
7,CHEBI:8899,rottlerin,HGNC:9399,PRKCD,0.312910
8,CHEBI:27958,cocaine,HGNC:3796,FOS,0.312421
9,CHEBI:15355,acetylcholine,HGNC:983,BCHE,0.311961


In [26]:
execute_example("MONDO:0009971","newborn respiratory distress syndrome")

Most enriched property: central_nervous_system_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.394311
1,CHEBI:27958,cocaine,HGNC:983,BCHE,0.356910
2,CHEBI:27958,cocaine,HGNC:11049,SLC6A3,0.314145
3,CHEBI:27732,caffeine,HGNC:10483,RYR1,0.312774
4,CHEBI:27958,cocaine,HGNC:3796,FOS,0.312421
5,CHEBI:8107,phenytoin,HGNC:1516,CAT,0.311990
6,CHEBI:27958,cocaine,HGNC:1033,BDNF,0.311847
7,CHEBI:3766,clozapine,HGNC:2596,CYP1A2,0.311640
8,CHEBI:2679,amphetamine,HGNC:11049,SLC6A3,0.310993
9,CHEBI:16236,ethanol,HGNC:1516,CAT,0.310948


In [27]:
execute_example("MONDO:0011565","metabolic syndrome X")

Most enriched property: psychotropic_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.252074
1,CHEBI:39867,valproic acid,HGNC:992,BCL2L1,0.232192
2,CHEBI:27732,caffeine,HGNC:10483,RYR1,0.173596
3,CHEBI:3766,clozapine,HGNC:2596,CYP1A2,0.172704
4,CHEBI:7640,nortriptyline,HGNC:2625,CYP2D6,0.171859
5,CHEBI:3766,clozapine,HGNC:3024,DRD3,0.171820
6,CHEBI:6809,methamphetamine,HGNC:10935,SLC18A2,0.171625
7,CHEBI:3387,carbamazepine,HGNC:10585,SCN1A,0.171434
8,CHEBI:6809,methamphetamine,HGNC:11049,SLC6A3,0.171353
9,CHEBI:28487,reserpine,HGNC:10935,SLC18A2,0.171330


In [28]:
execute_example("MONDO:0008752","Alexander disease")

No contributing chemicals


In [29]:
execute_example("MONDO:0008364","Raynaud disease")

Most enriched property: neurotransmitter_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16240,hydrogen peroxide,HGNC:1516,CAT,0.598193
1,CHEBI:16240,hydrogen peroxide,HGNC:11998,TP53,0.459067
2,CHEBI:16240,hydrogen peroxide,HGNC:4638,GSTP1,0.414626
3,CHEBI:16240,hydrogen peroxide,HGNC:4553,GPX1,0.404164
4,CHEBI:8093,phenylephrine,HGNC:7876,NOS3,0.358661
5,CHEBI:18243,dopamine,HGNC:6834,MAOB,0.320748
6,CHEBI:16240,hydrogen peroxide,HGNC:16753,PRDX6,0.320390
7,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.315602
8,CHEBI:16240,hydrogen peroxide,HGNC:4629,GSTA4,0.314808
9,CHEBI:16240,hydrogen peroxide,HGNC:7408,MT3,0.314786


In [30]:
execute_example("MONDO:0004975","Alzheimer disease")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.501340
1,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.452526
2,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.450380
3,CHEBI:28201,rotenone,HGNC:16369,PARK7,0.450135
4,CHEBI:41774,tamoxifen,HGNC:3430,ERBB2,0.427993
5,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.426359
6,CHEBI:15843,arachidonic acid,HGNC:613,APOE,0.424453
7,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.406887
8,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.406518
9,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.406496


In [31]:
execute_example("MONDO:0018859","Leigh disease")

Most enriched property: acyl_donor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:15538,phytanoyl-CoA,HGNC:8940,PHYH,0.121767
1,CHEBI:15532,myristoyl-CoA,HGNC:7857,NMT1,0.121474
2,CHEBI:15532,myristoyl-CoA,HGNC:7858,NMT2,0.121327
3,CHEBI:15532,myristoyl-CoA,HGNC:83,ACAA2,0.087940
4,CHEBI:28325,(S)-3-hydroxydecanoyl-CoA,HGNC:4800,HSD17B10,0.087940
5,CHEBI:15532,myristoyl-CoA,HGNC:82,ACAA1,0.087940
6,CHEBI:15532,myristoyl-CoA,HGNC:19691,MECR,0.087940
7,CHEBI:15532,myristoyl-CoA,HGNC:4803,HADHB,0.087940
8,CHEBI:28325,(S)-3-hydroxydecanoyl-CoA,HGNC:4799,HADH,0.087940
9,CHEBI:28325,(S)-3-hydroxydecanoyl-CoA,HGNC:3247,EHHADH,0.087940


In [32]:
execute_example("MONDO:0007041","apert syndrome")

No contributing chemicals


In [33]:
execute_example("MONDO:0020066","ehlers-danlos syndrome")

No contributing chemicals


In [34]:
execute_example("MONDO:0012819","diabetic ketoacidosis")

Most enriched property: central_nervous_system_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:27958,cocaine,HGNC:983,BCHE,0.357121
1,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.314917
2,CHEBI:8107,phenytoin,HGNC:1516,CAT,0.312132
3,CHEBI:16236,ethanol,HGNC:1516,CAT,0.311183
4,CHEBI:7640,nortriptyline,HGNC:2625,CYP2D6,0.310755
5,CHEBI:3647,chlorpromazine,HGNC:2625,CYP2D6,0.310047
6,CHEBI:39867,valproic acid,HGNC:8583,SERPINE1,0.309944
7,CHEBI:9753,troglitazone,HGNC:9237,PPARGC1A,0.309857
8,CHEBI:2679,amphetamine,HGNC:6204,JUN,0.309772
9,CHEBI:3647,chlorpromazine,HGNC:2631,CYP2E1,0.309734


In [35]:
execute_example("MONDO:0007127","diffuse idiopathic skeletal hyperostosis")

No contributing chemicals


In [36]:
execute_example("MONDO:0009971","newborn respiratory distress syndrome")

Most enriched property: central_nervous_system_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.394311
1,CHEBI:27958,cocaine,HGNC:983,BCHE,0.356910
2,CHEBI:27958,cocaine,HGNC:11049,SLC6A3,0.314145
3,CHEBI:27732,caffeine,HGNC:10483,RYR1,0.312774
4,CHEBI:27958,cocaine,HGNC:3796,FOS,0.312421
5,CHEBI:8107,phenytoin,HGNC:1516,CAT,0.311990
6,CHEBI:27958,cocaine,HGNC:1033,BDNF,0.311847
7,CHEBI:3766,clozapine,HGNC:2596,CYP1A2,0.311640
8,CHEBI:2679,amphetamine,HGNC:11049,SLC6A3,0.310993
9,CHEBI:16236,ethanol,HGNC:1516,CAT,0.310948


In [37]:
execute_example("MONDO:0008758","mitochondrial DNA depletion syndrome 4a")

No contributing chemicals


In [38]:
execute_example("MONDO:0015891","hypogonadotropic hypogonadism associated with")

No contributing chemicals


In [39]:
execute_example("MONDO:0020142","metabolic disease with dementia")

No contributing chemicals


In [40]:
execute_example("MONDO:0015135","primary immunodeficiency due to a genetic")

No contributing chemicals


In [41]:
execute_example("MONDO:0011382","sickle cell anemia")

Most enriched property: micronutrient
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:18248,iron atom,HGNC:4553,GPX1,0.325319
1,CHEBI:15422,ATP,HGNC:8537,P2RX7,0.321788
2,CHEBI:30052,copper(0),HGNC:9449,PRNP,0.314955
3,CHEBI:28694,copper atom,HGNC:9449,PRNP,0.314955
4,CHEBI:27363,zinc atom,HGNC:9605,PTGS2,0.312025
5,CHEBI:28694,copper atom,HGNC:7782,NFE2L2,0.309842
6,CHEBI:30052,copper(0),HGNC:7782,NFE2L2,0.309842
7,CHEBI:27013,tocopherol,HGNC:992,BCL2L1,0.309814
8,CHEBI:18291,manganese atom,HGNC:2625,CYP2D6,0.309757
9,CHEBI:30052,copper(0),HGNC:870,ATP7B,0.286539


In [42]:
execute_example("MONDO:0019037","progressive supranuclear palsy")

Most enriched property: antiparkinson_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:15765,L-dopa,HGNC:14581,PINK1,0.311546
1,CHEBI:15765,L-dopa,HGNC:4553,GPX1,0.233533
2,CHEBI:15765,L-dopa,HGNC:3415,EPO,0.233526
3,CHEBI:15765,L-dopa,HGNC:8512,OTC,0.199052
4,CHEBI:57504,L-dopa zwitterion,HGNC:14581,PINK1,0.197719
5,CHEBI:57504,L-dopa zwitterion,HGNC:8512,OTC,0.137548
6,CHEBI:57504,L-dopa zwitterion,HGNC:3415,EPO,0.137548
7,CHEBI:57504,L-dopa zwitterion,HGNC:4553,GPX1,0.137548


In [43]:
execute_example("MONDO:0012048","endogenous depression")

No contributing chemicals


In [44]:
execute_example("MONDO:0015789","non-acquired combined pituitary hormone defici.")

No contributing chemicals


In [45]:
execute_example("MONDO:0007972","Meniere disease")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.356981
1,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.327172
2,CHEBI:27827,paraoxon,HGNC:9204,PON1,0.322908
3,CHEBI:15843,arachidonic acid,HGNC:435,ALOX5,0.320437
4,CHEBI:44185,methotrexate,HGNC:7436,MTHFR,0.312687
5,CHEBI:44185,methotrexate,HGNC:7468,MTR,0.309959
6,CHEBI:44185,methotrexate,HGNC:1527,CAV1,0.309731
7,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.307062
8,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.306598
9,CHEBI:28201,rotenone,HGNC:16369,PARK7,0.305231


In [46]:
execute_example("MONDO:0009637","mitochondrial myopathy")

Most enriched property: anticholesteremic_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:9150,simvastatin,HGNC:5006,HMGCR,0.232698
1,CHEBI:63618,pravastatin,HGNC:5006,HMGCR,0.232305
2,CHEBI:3750,clofibrate,HGNC:9232,PPARA,0.232219
3,CHEBI:40303,lovastatin,HGNC:5464,IGF1,0.232204
4,CHEBI:40303,lovastatin,HGNC:17885,STEAP2,0.232198
5,CHEBI:40303,lovastatin,HGNC:23115,EAF2,0.232198
6,CHEBI:40303,lovastatin,HGNC:377,AKAP7,0.232198
7,CHEBI:40303,lovastatin,HGNC:14653,PARVB,0.232198
8,CHEBI:40303,lovastatin,HGNC:14450,LPIN2,0.232198
9,CHEBI:40303,lovastatin,HGNC:493,ANK2,0.232198


In [47]:
execute_example("MONDO:0007154","arteriovenous malformations of the brain")

Most enriched property: glutamate_transporter_activator
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:3387,carbamazepine,HGNC:10585,SCN1A,0.232923
1,CHEBI:3387,carbamazepine,HGNC:2637,CYP3A4,0.183771


In [48]:
execute_example("MONDO:0015977","agammaglobulinemia")

Most enriched property: immunosuppressive_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:46345,5-fluorouracil,HGNC:3012,DPYD,0.628871
1,CHEBI:46345,5-fluorouracil,HGNC:11998,TP53,0.451103
2,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.394836
3,CHEBI:41879,dexamethasone,HGNC:990,BCL2,0.356025
4,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.327492
5,CHEBI:46345,5-fluorouracil,HGNC:17297,RNF34,0.314798
6,CHEBI:46345,5-fluorouracil,HGNC:9955,RELA,0.314765
7,CHEBI:9168,sirolimus,HGNC:9588,PTEN,0.313517
8,CHEBI:46345,5-fluorouracil,HGNC:3236,EGFR,0.311933
9,CHEBI:28830,chlorambucil,HGNC:4628,GSTA3,0.311740


In [49]:
execute_example("MONDO:0017660","rare genetic parkinsonian disorder")

No contributing chemicals


In [50]:
execute_example("MONDO:0011827","patent ductus arteriosus")

Most enriched property: sex_hormone
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:17026,progesterone,HGNC:9605,PTGS2,0.311505
1,CHEBI:16469,17beta-estradiol,HGNC:11920,FAS,0.232311
2,CHEBI:16469,17beta-estradiol,HGNC:59,ABCC8,0.232217
3,CHEBI:41922,diethylstilbestrol,HGNC:7392,MSX2,0.232186
4,CHEBI:28088,genistein,HGNC:2561,CXCR4,0.232157
5,CHEBI:28088,genistein,HGNC:31635,MIR34A,0.197700
6,CHEBI:33216,bisphenol A,HGNC:7881,NOTCH1,0.197700
7,CHEBI:53578,hexamethylene diisocyanate,HGNC:7881,NOTCH1,0.197700
8,CHEBI:17347,testosterone,HGNC:2625,CYP2D6,0.182799
9,CHEBI:17026,progesterone,HGNC:2593,CYP17A1,0.182733


In [51]:
execute_example("MONDO:0008575","nicotine dependence")

Most enriched property: psychotropic_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.395201
1,CHEBI:3647,chlorpromazine,HGNC:2610,CYP2A6,0.312216
2,CHEBI:7640,nortriptyline,HGNC:2625,CYP2D6,0.312190
3,CHEBI:3766,clozapine,HGNC:2596,CYP1A2,0.311953
4,CHEBI:6809,methamphetamine,HGNC:11049,SLC6A3,0.311853
5,CHEBI:7640,nortriptyline,HGNC:11050,SLC6A4,0.311780
6,CHEBI:6809,methamphetamine,HGNC:10935,SLC18A2,0.311631
7,CHEBI:3766,clozapine,HGNC:3024,DRD3,0.311358
8,CHEBI:3647,chlorpromazine,HGNC:3023,DRD2,0.311337
9,CHEBI:6809,methamphetamine,HGNC:4632,GSTM1,0.311148


In [52]:
execute_example("MONDO:0000455","cone dystrophy")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.356981
1,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.307062
2,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.306598
3,CHEBI:28201,rotenone,HGNC:16369,PARK7,0.305231
4,CHEBI:41774,tamoxifen,HGNC:3430,ERBB2,0.285136
5,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.281753
6,CHEBI:49668,gefitinib,HGNC:651,AREG,0.279502
7,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.264167
8,CHEBI:64310,crizotinib,HGNC:427,ALK,0.263334
9,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.263048


In [53]:
execute_example("MONDO:0014131","hypohidrosis-enamel hypoplasia-palmoplantar ke...")

No contributing chemicals


In [54]:
execute_example("MONDO:0015547","genetic dementia")

No contributing chemicals


In [55]:
execute_example("MONDO:0015978","functional neutrophil defect")

No contributing chemicals


In [56]:
execute_example("MONDO:0007432","CADASIL")

No contributing chemicals


In [57]:
execute_example("MONDO:0011165","glomerulopathy with fibronectin deposits 2")

No contributing chemicals


In [58]:
execute_example("MONDO:0007089","Alzheimer's disease 2")

Most enriched property: micronutrient
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:30052,copper(0),HGNC:870,ATP7B,0.365313
1,CHEBI:28694,copper atom,HGNC:870,ATP7B,0.365313
2,CHEBI:15422,ATP,HGNC:8537,P2RX7,0.321856
3,CHEBI:30052,copper(0),HGNC:9449,PRNP,0.314955
4,CHEBI:28694,copper atom,HGNC:9449,PRNP,0.314955
5,CHEBI:27363,zinc atom,HGNC:3040,RCAN1,0.309974
6,CHEBI:30052,copper(0),HGNC:7996,NRF1,0.309809
7,CHEBI:28694,copper atom,HGNC:7996,NRF1,0.309809
8,CHEBI:27363,zinc atom,NCBIGENE:17748,MT1,0.281040
9,CHEBI:15422,ATP,HGNC:3942,MTOR,0.260472


In [59]:
execute_example("MONDO:0013099","combined pituitary hormone deficiencies, genet...")

No contributing chemicals


In [60]:
execute_example("MONDO:0005578","arthritis")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.466426
1,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.451458
2,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.450380
3,CHEBI:41774,tamoxifen,HGNC:3430,ERBB2,0.427993
4,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.426173
5,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.420899
6,CHEBI:44185,methotrexate,HGNC:53,ABCC2,0.414757
7,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.405370
8,CHEBI:27827,paraoxon,HGNC:9204,PON1,0.402188
9,CHEBI:15843,arachidonic acid,HGNC:435,ALOX5,0.401941


In [61]:
execute_example("MONDO:0005148","type 2 diabetes mellitus")

Most enriched property: central_nervous_system_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:6809,methamphetamine,HGNC:7872,NOS1,0.394137
1,CHEBI:27958,cocaine,HGNC:983,BCHE,0.356948
2,CHEBI:9753,troglitazone,HGNC:9237,PPARGC1A,0.330246
3,CHEBI:9753,troglitazone,HGNC:10901,SKP2,0.323830
4,CHEBI:9753,troglitazone,HGNC:10477,RXRA,0.323822
5,CHEBI:16236,ethanol,HGNC:13633,ADIPOQ,0.320726
6,CHEBI:16236,ethanol,HGNC:404,ALDH2,0.316814
7,CHEBI:16236,ethanol,HGNC:6018,IL6,0.314273
8,CHEBI:27958,cocaine,HGNC:11049,SLC6A3,0.314145
9,CHEBI:27958,cocaine,HGNC:1033,BDNF,0.312651


In [62]:
execute_example("MONDO:0002406","dermatitis")

Most enriched property: inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:45713,trans-resveratrol,HGNC:1504,CASP3,0.539311
1,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.501156
2,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.450380
3,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.450152
4,CHEBI:45713,trans-resveratrol,HGNC:14929,SIRT1,0.448350
5,CHEBI:41774,tamoxifen,HGNC:3430,ERBB2,0.427993
6,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.425416
7,CHEBI:64310,crizotinib,HGNC:427,ALK,0.405664
8,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.405370
9,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.405261


In [63]:
execute_example("MONDO:0008383","rheumatoid arthritis")

Most enriched property: antirheumatic_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.565116
1,CHEBI:44185,methotrexate,HGNC:53,ABCC2,0.509554
2,CHEBI:44185,methotrexate,HGNC:7436,MTHFR,0.462421
3,CHEBI:44185,methotrexate,HGNC:10937,SLC19A1,0.460292
4,CHEBI:44185,methotrexate,HGNC:74,ABCG2,0.458744
5,CHEBI:44185,methotrexate,HGNC:2891,DKK1,0.458217
6,CHEBI:44185,methotrexate,HGNC:1071,BMP4,0.458021
7,CHEBI:44185,methotrexate,HGNC:12441,TYMS,0.457990
8,CHEBI:44185,methotrexate,HGNC:10630,CCL4,0.457899
9,CHEBI:44185,methotrexate,HGNC:10472,RUNX2,0.457507


In [64]:
execute_example("MONDO:0005359","drug-induced liver injury")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.501183
1,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.450538
2,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.449990
3,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.425412
4,CHEBI:64310,crizotinib,HGNC:427,ALK,0.405709
5,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.405646
6,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.405297
7,CHEBI:64310,crizotinib,HGNC:3236,EGFR,0.397590
8,CHEBI:41774,tamoxifen,HGNC:3236,EGFR,0.396390
9,CHEBI:46195,paracetamol,HGNC:4311,GCLC,0.394594


In [65]:
execute_example("MONDO:0006573","lipodystrophy (disease)")

Most enriched property: antiviral_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:10110,zidovudine,HGNC:11830,TK1,0.360522
1,CHEBI:10110,zidovudine,HGNC:11180,SOD2,0.314147
2,CHEBI:10110,zidovudine,HGNC:1516,CAT,0.314130
3,CHEBI:175901,gemcitabine,HGNC:11283,SRC,0.277389
4,CHEBI:60761,floxuridine,HGNC:882,ATR,0.277185
5,CHEBI:10110,zidovudine,HGNC:12816,XPC,0.236011
6,CHEBI:28680,cytarabine,HGNC:2704,DCK,0.233707
7,CHEBI:175901,gemcitabine,HGNC:11003,SLC29A1,0.233477
8,CHEBI:175901,gemcitabine,HGNC:11998,TP53,0.233064
9,CHEBI:28680,cytarabine,HGNC:11003,SLC29A1,0.232559


In [66]:
execute_example("MONDO:0008226","aggressive periodontitis")

No contributing chemicals


In [67]:
execute_example("MONDO:0001627","dementia")

Most enriched property: neurotransmitter_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16240,hydrogen peroxide,HGNC:1516,CAT,0.598193
1,CHEBI:16240,hydrogen peroxide,HGNC:11998,TP53,0.539583
2,CHEBI:16240,hydrogen peroxide,HGNC:4638,GSTP1,0.494836
3,CHEBI:16240,hydrogen peroxide,HGNC:4553,GPX1,0.484513
4,CHEBI:18243,dopamine,HGNC:6834,MAOB,0.416153
5,CHEBI:16240,hydrogen peroxide,HGNC:16753,PRDX6,0.399727
6,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.394856
7,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.374726
8,CHEBI:15355,acetylcholine,HGNC:613,APOE,0.367278
9,CHEBI:8093,phenylephrine,HGNC:7876,NOS3,0.358576


In [68]:
execute_example("MONDO:0005265","inflammatory bowel disease")

Most enriched property: neurotransmitter_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16240,hydrogen peroxide,HGNC:1516,CAT,0.598193
1,CHEBI:16240,hydrogen peroxide,HGNC:11998,TP53,0.539739
2,CHEBI:16240,hydrogen peroxide,HGNC:4638,GSTP1,0.494836
3,CHEBI:16240,hydrogen peroxide,HGNC:4553,GPX1,0.484617
4,CHEBI:16240,hydrogen peroxide,HGNC:16753,PRDX6,0.399688
5,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.397610
6,CHEBI:16240,hydrogen peroxide,HGNC:7408,MT3,0.394007
7,CHEBI:8093,phenylephrine,HGNC:7873,NOS2,0.360076
8,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.359543
9,CHEBI:8093,phenylephrine,HGNC:7876,NOS3,0.358774


In [69]:
execute_example("MONDO:0004979","asthma")

Most enriched property: neurotransmitter_agent
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:16240,hydrogen peroxide,HGNC:1516,CAT,0.616153
1,CHEBI:16240,hydrogen peroxide,HGNC:11998,TP53,0.539583
2,CHEBI:16240,hydrogen peroxide,HGNC:4638,GSTP1,0.497160
3,CHEBI:16240,hydrogen peroxide,HGNC:4553,GPX1,0.484400
4,CHEBI:41879,dexamethasone,HGNC:6018,IL6,0.421916
5,CHEBI:2549,albuterol,HGNC:286,ADRB2,0.419321
6,CHEBI:16240,hydrogen peroxide,HGNC:16753,PRDX6,0.399654
7,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.383716
8,CHEBI:41879,dexamethasone,HGNC:990,BCL2,0.381258
9,CHEBI:8093,phenylephrine,HGNC:7873,NOS2,0.359049


In [70]:
execute_example("MONDO:0005298","osteoporosis")

Most enriched property: enzyme_inhibitor
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:4911,etoposide,HGNC:51,ABCC1,0.501156
1,CHEBI:41774,tamoxifen,HGNC:11998,TP53,0.464564
2,CHEBI:5130,flurbiprofen,HGNC:11998,TP53,0.449926
3,CHEBI:41774,tamoxifen,HGNC:3430,ERBB2,0.442085
4,CHEBI:46195,paracetamol,HGNC:7782,NFE2L2,0.424533
5,CHEBI:41774,tamoxifen,HGNC:2625,CYP2D6,0.419356
6,CHEBI:41774,tamoxifen,HGNC:3236,EGFR,0.410206
7,CHEBI:44185,methotrexate,HGNC:2861,DHFR,0.408148
8,CHEBI:15343,acetaldehyde,HGNC:404,ALDH2,0.405362
9,CHEBI:27827,paraoxon,HGNC:9204,PON1,0.402235


In [ ]:
execute_example("MONDO:0006932","pulmonary edema")

Most enriched property: cardiovascular_drug
Return Status: 200


,n0 - id,n0 - name,n1 - id,n1 - name,score
0,CHEBI:8093,phenylephrine,HGNC:7876,NOS3,0.360030
1,CHEBI:15355,acetylcholine,HGNC:333,AGT,0.359561
2,CHEBI:8093,phenylephrine,HGNC:7873,NOS2,0.358519
3,CHEBI:27958,cocaine,HGNC:983,BCHE,0.357878
4,CHEBI:18243,dopamine,HGNC:6834,MAOB,0.320878
5,CHEBI:16480,nitric oxide,HGNC:333,AGT,0.317706
6,CHEBI:34665,debrisoquin,HGNC:2625,CYP2D6,0.313599
7,CHEBI:27958,cocaine,HGNC:3796,FOS,0.313367
8,CHEBI:28787,nitroglycerin,HGNC:2578,CYBB,0.313315
9,CHEBI:17245,carbon monoxide,HGNC:333,AGT,0.312877


In [ ]:
execute_example("MONDO:0005180","Parkinson disease")

In [ ]:
execute_example("MONDO:0006658","arteriolosclerosis")

In [ ]:
execute_example("MONDO:0005090","schizophrenia")

In [ ]:
execute_example("MONDO:0005152","hypopituitarism")

In [ ]:
execute_example("MONDO:0001835","facial paralysis")

In [ ]:
execute_example("MONDO:0001824","polyneuropathy")

In [ ]:
execute_example("MONDO:0005301","multiple sclerosis")

In [ ]:
execute_example("MONDO:0006644","alcoholic liver cirrhosis")